# Setup
We need to get everything set up.

In [ ]:
# %pip install https://huggingface.co/easyh/de_fnhd_nerdh/resolve/main/de_fnhd_nerdh-any-py3-none-any.whl
!python -m spacy download "de_core_news_lg"
%pip install pyldavis
%pip install gensim

In [1]:
import spacy
from pathlib import Path
import gensim
import pyLDAvis
from pyLDAvis import gensim_models as gsv
from constants import STOPWORDS as stops

In [2]:
# nlp = spacy.load("de_fnhd_nerdh")
nlp = spacy.load("de_core_news_lg")

/Users/kraus0001/opt/anaconda3/envs/new3x/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/kraus0001/opt/anaconda3/envs/new3x/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/kraus0001/opt/anaconda3/envs/new3x/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/kraus0001/opt/anaconda3/envs/new3x/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(s

In [3]:
input_files = Path("./input/").rglob("*.txt")

In [4]:
full_corpus = {}

for i in input_files:
    with open(i, "r+") as f:
        txt = f.read()
    book_name = str(i).split("/")[-1]
    book_name = book_name.split(".")[0]
    full_corpus[book_name] = txt
print(len(full_corpus))

6


In [5]:
by_chapters = {}

for k, v in full_corpus.items():
    chaps = v.split("Capitel")
    chap_num = 0
    for i in chaps:
        curr_chap = f"{k}-{chap_num}-Kap"
        i = i.replace("=\n", "")
        by_chapters[curr_chap] = i
        chap_num += 1
print(len(by_chapters))

397


In [6]:
docs_list: list[list[str]] = []
for i in nlp.pipe(by_chapters.values(), n_process=7):
    itoks = [token.lemma_.lower() for token in i if token.lower_ not in stops and "\n" not in token.lower_ and not token.is_stop]
    docs_list.append(itoks)

/Users/kraus0001/opt/anaconda3/envs/new3x/lib/python3.10/site-packages/catalogue/__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/kraus0001/opt/anaconda3/envs/new3x/lib/python3.10/site-packages/catalogue/__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/kraus0001/opt/anaconda3/envs/new3x/lib/python3.10/site-packages/catalogue/__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/kraus0001/opt/anaconda3/envs/new3x/lib/python3.10/site-packages/catalogue/__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(s

In [7]:
corpus_dictionary = gensim.corpora.Dictionary(docs_list)
analysis_corpus = [corpus_dictionary.doc2bow(i) for i in docs_list]
lda_model = gensim.models.ldamulticore.LdaMulticore(corpus=analysis_corpus,
                                              id2word=corpus_dictionary,
                                              iterations=50,
                                              num_topics=10,
                                              workers = 7,
                                              passes=10)


In [8]:
lda_display = gsv.prepare(lda_model, analysis_corpus, corpus_dictionary)
pyLDAvis.display(lda_display)

/Users/kraus0001/opt/anaconda3/envs/new3x/lib/python3.10/site-packages/pyLDAvis/_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
